# TFX Components Walk-through

## Learning Objectives

1.  Develop a high level understanding of TFX pipeline components.
2.  Learn how to use a TFX Interactive Context for prototype development of TFX pipelines.
3.  Work with the Tensorflow Data Validation (TFDV) library to check and analyze input data.
4.  Utilize the Tensorflow Transform (TFT) library for scalable data preprocessing and feature transformations.
5.  Employ the Tensorflow Model Analysis (TFMA) library for model evaluation.

In this lab, you will work with the [Covertype Data Set](https://github.com/jarokaz/mlops-labs/blob/master/datasets/covertype/README.md) and use TFX to analyze, understand, and pre-process the dataset and train, analyze, validate, and deploy a multi-class classification model to predict the type of forest cover from cartographic features.

You will utilize  **TFX Interactive Context** to work with the TFX components interactivelly in a Jupyter notebook environment. Working in an interactive notebook is useful when doing initial data exploration, experimenting with models, and designing ML pipelines. You should be aware that there are differences in the way interactive notebooks are orchestrated, and how they access metadata artifacts. In a production deployment of TFX on GCP, you will use Managed Pipelines to ochestrate the flow and run components. In an interactive mode, the notebook itself is the orchestrator, running each TFX component as you execute the notebook cells. In a production deployment, ML Metadata will be managed in Managed ML Metadata service, and artifacts in apersistent store such as Google Cloud Storage. In an interactive mode, both properties and payloads are stored in a local file system of the Jupyter host.

**Setup Note:**
Currently, TFMA visualizations do not render properly in JupyterLab. It is recommended to run this notebook in Jupyter Classic Notebook. To switch to Classic Notebook select *Launch Classic Notebook* from the *Help* menu.

In [61]:
import absl
import os
import tempfile
import time

import tensorflow as tf
import tensorflow_data_validation as tfdv
import tensorflow_model_analysis as tfma
import tensorflow_transform as tft
import tfx

from pprint import pprint
from tensorflow_metadata.proto.v0 import schema_pb2, statistics_pb2, anomalies_pb2
from tensorflow_transform.tf_metadata import schema_utils

from ml_metadata.metadata_store import metadata_store
from ml_metadata.proto import metadata_store_pb2

from tfx.components import CsvExampleGen
from tfx.components import Evaluator
from tfx.components import ExampleValidator
from tfx.components import InfraValidator
from tfx.components import Pusher
from tfx.components import ResolverNode
from tfx.components import SchemaGen
from tfx.components import StatisticsGen
from tfx.components import Trainer
from tfx.components import Transform
from tfx.components.base import executor_spec
from tfx.components.common_nodes.importer_node import ImporterNode
from tfx.components.trainer import executor as trainer_executor

from tfx.dsl.experimental import latest_blessed_model_resolver

from tfx.orchestration.beam.beam_dag_runner import BeamDagRunner
from tfx.orchestration.metadata import sqlite_metadata_connection_config
from tfx.orchestration.pipeline import Pipeline
from tfx.orchestration import metadata

from tfx.proto import evaluator_pb2
from tfx.proto import example_gen_pb2
from tfx.proto import infra_validator_pb2
from tfx.proto import pusher_pb2
from tfx.proto import trainer_pb2
from tfx.proto.evaluator_pb2 import SingleSlicingSpec

from tfx.utils.dsl_utils import external_input

from tfx.types import Channel
from tfx.types.standard_artifacts import Examples
from tfx.types.standard_artifacts import ExampleStatistics
from tfx.types.standard_artifacts import ExampleAnomalies
from tfx.types.standard_artifacts import Model
from tfx.types.standard_artifacts import ModelEvaluation
from tfx.types.standard_artifacts import ModelBlessing
from tfx.types.standard_artifacts import InfraBlessing

## Set up the environment
### Verify TFX SDK Version

**Note**: this lab was developed and tested with the following TF ecosystem package versions:

`Tensorflow Version: 2.3.0`  
`TFX Version: 0.23.0.caip20200818`  
`TFDV Version: 0.23.0`  
`TFMA Version: 0.23.0`

If you encounter errors with the above imports (e.g. TFX component not found), check your package versions in the cell below.

In [2]:
print("Tensorflow Version:", tf.__version__)
print("TFX Version:", tfx.__version__)
print("TFDV Version:", tfdv.__version__)
print("TFMA Version:", tfma.VERSION_STRING)

absl.logging.set_verbosity(absl.logging.INFO)

Tensorflow Version:2.3.0
TFX Version:0.23.0.caip20200818
TFDV Version:0.23.0
TFMA Version:0.23.0


If the versions above do not match, update your packages in the current Jupyter kernel. 

### Update `PATH` with the location of TFX SDK.

In [3]:
os.environ['PATH'] += os.pathsep + '/home/jupyter/.local/bin'

## Configure lab settings

Set constants, location paths and other environment settings. 

### Settings for Beam runner

In [4]:
LOCAL_PIPELINE_ROOT= os.path.join(os.sep, 'home', 'jupyter', 'pipeline-root')
METADATA_PATH = os.path.join(LOCAL_PIPELINE_ROOT, 'metadata.sqlite')
SERVING_MODEL_DIR =os.path.join(os.sep, 'home', 'jupyter', 'serving_model')
metadata_connection_config=sqlite_metadata_connection_config(METADATA_PATH)

### Settings for Managed Pipelines runner

In [5]:
PROJECT_ID = 'mlops-dev-env'
GCS_BUCKET_NAME = 'mlops-dev-workspace'
GCS_PIPELINE_ROOT = f'gs://{GCS_BUCKET_NAME}/pipeline-root'
BASE_IMAGE = f'gcr.io/caip-pipelines-assets/tfx:{tfx.__version__}'

### Source dataset

In [7]:
DATA_ROOT = 'gs://workshop-datasets/covertype/small'

## Configure TFX Pipeline

### Ingesting data using ExampleGen

In any ML development process the first step  is to ingest the training and test datasets. The `ExampleGen` component ingests data into a TFX pipeline. It consumes external files/services to generate a set file files in the `TFRecord` format,  which will be used by other TFX components. It can also shuffle the data and split into an arbitrary number of partitions.

<img src=https://github.com/GoogleCloudPlatform/mlops-on-gcp/raw/master/images/ExampleGen.png width="300">


In this exercise, you use the `CsvExampleGen` specialization of `ExampleGen` to ingest CSV files from a GCS location. The component is configured to split the input data into two splits - `train` and `eval` - using 4:1 ratio.  

In [8]:
output_config = example_gen_pb2.Output(
    split_config=example_gen_pb2.SplitConfig(splits=[
        example_gen_pb2.SplitConfig.Split(name='train', hash_buckets=4),
        example_gen_pb2.SplitConfig.Split(name='eval', hash_buckets=1)
    ]))

example_gen = tfx.components.CsvExampleGen(
    instance_name='Data_Extraction_Spliting',
    input_base=DATA_ROOT,
    output_config=output_config
)

#### Create the pipeline

In [9]:
pipeline_name = 'example_gen_only'
components = [example_gen]

pipeline = Pipeline(
    pipeline_name=pipeline_name,
    pipeline_root=f'{LOCAL_PIPELINE_ROOT}/{pipeline_name}',
    metadata_connection_config=metadata_connection_config,
    components=components
)

#### Run the pipeline

In [10]:
BeamDagRunner().run(pipeline)

INFO:absl:Component CsvExampleGen.Data_Extraction_Spliting depends on [].
INFO:absl:Component CsvExampleGen.Data_Extraction_Spliting is scheduled.
INFO:absl:Component CsvExampleGen.Data_Extraction_Spliting is running.
INFO:absl:Running driver for CsvExampleGen.Data_Extraction_Spliting
INFO:absl:MetadataStore with DB connection initialized
INFO:absl:select span and version = (0, None)
INFO:absl:latest span and version = (0, None)
INFO:absl:Running executor for CsvExampleGen.Data_Extraction_Spliting
INFO:absl:Generating examples.
INFO:absl:Processing input csv data gs://workshop-datasets/covertype/small/* to TFExample.
INFO:absl:Examples generated.
INFO:absl:Running publisher for CsvExampleGen.Data_Extraction_Spliting
INFO:absl:MetadataStore with DB connection initialized
INFO:absl:Component CsvExampleGen.Data_Extraction_Spliting is finished.


#### Examine artifacts generated by the pipeline run

In [11]:
connection_config = metadata_store_pb2.ConnectionConfig()
connection_config.sqlite.filename_uri = METADATA_PATH
connection_config.sqlite.connection_mode = 3 # READWRITE_OPENCREATE

with metadata.Metadata(connection_config) as store:
    examples_artifacts = store.get_artifacts_by_type(Examples.TYPE_NAME)

for element in examples_artifacts:
    print(element.uri)
    print(element.properties)

INFO:absl:MetadataStore with DB connection initialized
/home/jupyter/pipeline-root/example_gen_only/CsvExampleGen.Data_Extraction_Spliting/examples/1
{'split_names': string_value: "[\"train\", \"eval\"]"
}
/home/jupyter/pipeline-root/example_gen_only/CsvExampleGen.Data_Extraction_Spliting/examples/2
{'split_names': string_value: "[\"train\", \"eval\"]"
}
/home/jupyter/pipeline-root/example_gen_only/CsvExampleGen.Data_Extraction_Spliting/examples/3
{'split_names': string_value: "[\"train\", \"eval\"]"
}


#### Examine the ingested data

In [15]:
examples_uri = examples_artifacts[-1].uri
tfrecord_filenames = [os.path.join(examples_uri, 'train', name)
                      for name in os.listdir(os.path.join(examples_uri, 'train'))]
dataset = tf.data.TFRecordDataset(tfrecord_filenames, compression_type="GZIP")
for tfrecord in dataset.take(2):
  example = tf.train.Example()
  example.ParseFromString(tfrecord.numpy())
  for name, feature in example.features.feature.items():
    if feature.HasField('bytes_list'):
        value = feature.bytes_list.value
    if feature.HasField('float_list'):
        value = feature.float_list.value
    if feature.HasField('int64_list'):
        value = feature.int64_list.value
    print('{}: {}'.format(name, value))
  print('******')

Horizontal_Distance_To_Roadways: [757]
Elevation: [3142]
Slope: [9]
Wilderness_Area: [b'Commanche']
Vertical_Distance_To_Hydrology: [101]
Cover_Type: [1]
Soil_Type: [b'C7757']
Hillshade_3pm: [157]
Horizontal_Distance_To_Fire_Points: [1871]
Hillshade_Noon: [247]
Hillshade_9am: [223]
Aspect: [183]
Horizontal_Distance_To_Hydrology: [648]
******
Horizontal_Distance_To_Fire_Points: [451]
Hillshade_Noon: [227]
Hillshade_9am: [245]
Aspect: [124]
Horizontal_Distance_To_Hydrology: [60]
Slope: [16]
Horizontal_Distance_To_Roadways: [124]
Elevation: [1967]
Wilderness_Area: [b'Cache']
Vertical_Distance_To_Hydrology: [9]
Soil_Type: [b'C2704']
Cover_Type: [2]
Hillshade_3pm: [105]
******


### Generating statistics using StatisticsGen

The `StatisticsGen`  component generates data statistics that can be used by other TFX components. StatisticsGen uses [TensorFlow Data Validation](https://www.tensorflow.org/tfx/data_validation/get_started). `StatisticsGen` generate statistics for each split in the `ExampleGen` component's output. In our case there two splits: `train` and `eval`.

<img src=https://github.com/GoogleCloudPlatform/mlops-on-gcp/raw/master/images/StatisticsGen.png width="200">

In [22]:
statistics_gen = tfx.components.StatisticsGen(
    instance_name='Statistics_Generation',
    examples=example_gen.outputs['examples'])

INFO:absl:Excluding no splits because exclude_splits is not set.


### Infering data schema using SchemaGen

Some TFX components use a description input data called a schema. The schema is an instance of `schema.proto`. It can specify data types for feature values, whether a feature has to be present in all examples, allowed value ranges, and other properties. `SchemaGen` automatically generates the schema by inferring types, categories, and ranges from data statistics. The auto-generated schema is best-effort and only tries to infer basic properties of the data. It is expected that developers review and modify it as needed. `SchemaGen` uses [TensorFlow Data Validation](https://www.tensorflow.org/tfx/data_validation/get_started).

The `SchemaGen` component generates the schema using the statistics for the `train` split. The statistics for other splits are ignored.

<img src=https://github.com/GoogleCloudPlatform/mlops-on-gcp/raw/master/images/SchemaGen.png width="200">

In [25]:
schema_gen = SchemaGen(
    statistics=statistics_gen.outputs['statistics'],
    infer_feature_shape=False)

INFO:absl:Excluding no splits because exclude_splits is not set.


### Validating data with ExampleValidator

The `ExampleValidator` component identifies anomalies in data.  It identifies anomalies by comparing data statistics computed by the `StatisticsGen` component against a schema generated by `SchemaGen` or imported by `ImporterNode`.

`ExampleValidator` can detect different classes of anomalies. For example it can:

- perform validity checks by comparing data statistics against a schema 
- detect training-serving skew by comparing training and serving data.
- detect data drift by looking at a series of data.


The `ExampleValidator` component validates the data in the `eval` split only. Other splits are ignored. 

<img src=https://github.com/GoogleCloudPlatform/mlops-on-gcp/raw/master/images/ExampleValidator.png width="350">

In [28]:
example_validator = ExampleValidator(
    statistics=statistics_gen.outputs['statistics'],
    schema=schema_gen.outputs['schema'],
    instance_name="Data_Validation"
)

INFO:absl:Excluding no splits because exclude_splits is not set.


### Preprocessing data with Transform

The `Transform` component performs data transformation and feature engineering. The `Transform` component consumes `tf.Examples` emitted from the `ExampleGen` component and emits the transformed feature data and the `SavedModel` graph that was used to process the data. The emitted `SavedModel`  can then be used by serving components to make sure that the same data pre-processing logic is applied at training and serving.

The `Transform` component requires more code than many other components because of the arbitrary complexity of the feature engineering that you may need for the data and/or model that you're working with. It requires code files to be available which define the processing needed.

<img src=https://github.com/GoogleCloudPlatform/mlops-on-gcp/raw/master/images/Transform.png width="400">

#### Define the pre-processing module

To configure `Trainsform`, you need to encapsulate your pre-processing code in the Python `preprocessing_fn` function and save it to a  python module that is then provided to the Transform component as an input. This module will be loaded by transform and the `preprocessing_fn` function will be called when the `Transform` component runs.

In most cases, your implementation of the `preprocessing_fn` makes extensive use of [TensorFlow Transform](https://www.tensorflow.org/tfx/guide/tft) for performing feature engineering on your dataset.

In [29]:
TRANSFORM_MODULE = 'preprocessing.py'

In [30]:
%%writefile {FEATURES_MODULE}
# Copyright 2020 Google LLC. All Rights Reserved.
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     http://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.
"""Covertype model  taxi model features."""
import tensorflow as tf
import tensorflow_model_analysis as tfma
import tensorflow_transform as tft
from tensorflow_transform.tf_metadata import schema_utils

NUMERIC_FEATURE_KEYS = [
    'Elevation', 'Aspect', 'Slope', 'Horizontal_Distance_To_Hydrology',
    'Vertical_Distance_To_Hydrology', 'Horizontal_Distance_To_Roadways',
    'Hillshade_9am', 'Hillshade_Noon', 'Hillshade_3pm',
    'Horizontal_Distance_To_Fire_Points'
]

CATEGORICAL_FEATURE_KEYS = ['Wilderness_Area', 'Soil_Type']

LABEL_KEY = 'Cover_Type'
NUM_CLASSES = 7

def transformed_name(key):
  return key + '_xf'

Writing {FEATURES_MODULE}


In [31]:
FEATURES_MODULE = 'features.py'

In [32]:
%%writefile {TRANSFORM_MODULE}
# Copyright 2020 Google LLC. All Rights Reserved.
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     http://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.
"""Covertype preprocessing.
This file defines a template for TFX Transform component.
"""

import tensorflow as tf
import tensorflow_transform as tft

import features

def _fill_in_missing(x):
  """Replace missing values in a SparseTensor.
  Fills in missing values of `x` with '' or 0, and converts to a dense tensor.
  Args:
    x: A `SparseTensor` of rank 2.  Its dense shape should have size at most 1
      in the second dimension.
  Returns:
    A rank 1 tensor where missing values of `x` have been filled in.
  """
  default_value = '' if x.dtype == tf.string else 0
  return tf.squeeze(
      tf.sparse.to_dense(
          tf.SparseTensor(x.indices, x.values, [x.dense_shape[0], 1]),
          default_value),
      axis=1)

def preprocessing_fn(inputs):
  """Preprocesses Covertype Dataset."""

  outputs = {}

  # Scale numerical features
  for key in features.NUMERIC_FEATURE_KEYS:
    outputs[features.transformed_name(key)] = tft.scale_to_z_score(
        _fill_in_missing(inputs[key]))

  # Generate vocabularies and maps categorical features
  for key in features.CATEGORICAL_FEATURE_KEYS:
    outputs[features.transformed_name(key)] = tft.compute_and_apply_vocabulary(
        x=_fill_in_missing(inputs[key]), num_oov_buckets=1, vocab_filename=key)

  # Convert Cover_Type to dense tensor
  outputs[features.transformed_name(features.LABEL_KEY)] = _fill_in_missing(
      inputs[features.LABEL_KEY])

  return outputs


Overwriting preprocessing.py


Configure the component.

In [34]:
transform = Transform(
    examples=example_gen.outputs['examples'],
    schema=schema_gen.outputs['schema'],
    module_file=TRANSFORM_MODULE)

### Train with the `Trainer` component

The `Trainer` component trains a model using TensorFlow.

`Trainer` takes:

- tf.Examples used for training and eval.
- A user provided module file that defines the trainer logic.
- A data schema created by `SchemaGen` or imported by `ImporterNode`.
- A proto definition of train args and eval args.
- An optional transform graph produced by upstream Transform component.
- An optional base models used for scenarios such as warmstart.

<img src=https://github.com/GoogleCloudPlatform/mlops-on-gcp/raw/master/images/Trainer.png width="400">


#### Define the trainer module

To configure `Trainer`, you need to encapsulate your training code in a Python module that is then provided to the `Trainer` as an input. 


In [36]:
TRAINER_MODULE_FILE = 'model.py'

In [55]:
%%writefile {TRAINER_MODULE_FILE}
# Copyright 2020 Google LLC. All Rights Reserved.
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     http://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.
"""The Covertype classifier DNN keras model."""

import absl
import os

import tensorflow as tf
import tensorflow_model_analysis as tfma
import tensorflow_transform as tft
from tensorflow_transform.tf_metadata import schema_utils

import features

HIDDEN_UNITS = [16, 8]
LEARNING_RATE = 0.001
TRAIN_BATCH_SIZE = 64
EVAL_BATCH_SIZE = 64

LOCAL_LOG_DIR = '/tmp/logs'


def _gzip_reader_fn(filenames):
  """Small utility returning a record reader that can read gzip'ed files."""
  return tf.data.TFRecordDataset(filenames, compression_type='GZIP')


def _get_serve_tf_examples_fn(model, tf_transform_output):
  """Returns a function that parses a serialized tf.Example and applies TFT."""

  model.tft_layer = tf_transform_output.transform_features_layer()

  @tf.function
  def serve_tf_examples_fn(serialized_tf_examples):
    """Returns the output to be used in the serving signature."""
    feature_spec = tf_transform_output.raw_feature_spec()
    feature_spec.pop(features.LABEL_KEY)
    parsed_features = tf.io.parse_example(serialized_tf_examples, feature_spec)

    transformed_features = model.tft_layer(parsed_features)

    return model(transformed_features)

  return serve_tf_examples_fn


def _input_fn(file_pattern, tf_transform_output, batch_size=200):
  """Generates features and label for tuning/training.
  Args:
    file_pattern: input tfrecord file pattern.
    tf_transform_output: A TFTransformOutput.
    batch_size: representing the number of consecutive elements of returned
      dataset to combine in a single batch
  Returns:
    A dataset that contains (features, indices) tuple where features is a
      dictionary of Tensors, and indices is a single Tensor of label indices.
  """
  transformed_feature_spec = (
      tf_transform_output.transformed_feature_spec().copy())

  dataset = tf.data.experimental.make_batched_features_dataset(
      file_pattern=file_pattern,
      batch_size=batch_size,
      features=transformed_feature_spec,
      reader=_gzip_reader_fn,
      label_key=features.transformed_name(features.LABEL_KEY))

  return dataset

def _build_keras_model(tf_transform_output, hidden_units, learning_rate):
  """Creates a DNN Keras model for classifying taxi data.
  Args:
    hidden_units: [int], the layer sizes of the DNN (input layer first).
  Returns:
    A keras Model.
  """

  numeric_columns = [
      tf.feature_column.numeric_column(
          key=features.transformed_name(key), 
          shape=())
      for key in features.NUMERIC_FEATURE_KEYS
  ]

  categorical_columns = [
      tf.feature_column.categorical_column_with_identity(
          key=features.transformed_name(key), 
          num_buckets=tf_transform_output.num_buckets_for_transformed_feature(features.transformed_name(key)), 
          default_value=0)
      for key in features.CATEGORICAL_FEATURE_KEYS
  ]

  indicator_columns = [
      tf.feature_column.indicator_column(categorical_column)
      for categorical_column in categorical_columns
  ]

  model = _wide_and_deep_classifier(
      # TODO(b/139668410) replace with premade wide_and_deep keras model
      wide_columns=indicator_columns,
      deep_columns=numeric_columns,
      dnn_hidden_units=hidden_units,
      learning_rate=learning_rate)
  return model


def _wide_and_deep_classifier(wide_columns, deep_columns, dnn_hidden_units, learning_rate):
  """Builds a simple keras wide and deep model.
  Args:
    wide_columns: Feature columns wrapped in indicator_column for wide (linear)
      part of the model.
    deep_columns: Feature columns for deep part of the model.
    dnn_hidden_units: [int], the layer sizes of the hidden DNN.
  Returns:
    A Wide and Deep Keras model
  """
  
  input_layers = {
      column.key: tf.keras.layers.Input(name=column.key, shape=(), dtype=tf.float32)
      for column in deep_columns
  }
  
  input_layers.update({
      column.categorical_column.key: tf.keras.layers.Input(name=column.categorical_column.key, shape=(), dtype=tf.int32)
      for column in wide_columns
  })
    
  deep = tf.keras.layers.DenseFeatures(deep_columns)(input_layers)
  for numnodes in dnn_hidden_units:
    deep = tf.keras.layers.Dense(numnodes)(deep)
  wide = tf.keras.layers.DenseFeatures(wide_columns)(input_layers)

  output = tf.keras.layers.Dense(features.NUM_CLASSES, activation='softmax')(
               tf.keras.layers.concatenate([deep, wide]))

  model = tf.keras.Model(input_layers, output)
  model.compile(
      loss='sparse_categorical_crossentropy',
      optimizer=tf.keras.optimizers.Adam(lr=learning_rate),
      metrics=[tf.keras.metrics.SparseCategoricalAccuracy()])
  model.summary(print_fn=absl.logging.info)
  return model

def _copy_tensorboard_logs(local_path, saved_model_path):
    """Copies Tensorboard logs to the subfolder in the GCS SavedModel location."""

    if saved_model_path[0:5] == 'gs://':
        pattern = '{}/*/events.out.tfevents.*'.format(local_path)
        dest_path = saved_model_path.rstrip('/') + '/' + 'logs'
        local_files = tf.io.gfile.glob(pattern)
        dest_log_files = [local_file.replace(local_path, dest_path) for local_file in local_files]
        for local_file, dest_file in zip(local_files, dest_log_files):
            tf.io.gfile.copy(local_file, dest_file)

# TFX Trainer will call this function.
def run_fn(fn_args):
  """Trains a model based on given args.
  Args:
    fn_args: Holds args used to train the model as name/value pairs.
  """
  
  tf_transform_output = tft.TFTransformOutput(fn_args.transform_output)
    
  train_dataset = _input_fn(fn_args.train_files, tf_transform_output, TRAIN_BATCH_SIZE)
  eval_dataset = _input_fn(fn_args.eval_files, tf_transform_output, EVAL_BATCH_SIZE)
    
  model = _build_keras_model(
      tf_transform_output=tf_transform_output,
      hidden_units=HIDDEN_UNITS,
      learning_rate=LEARNING_RATE
  )

  tensorboard_callback = tf.keras.callbacks.TensorBoard(
      log_dir=LOCAL_LOG_DIR, update_freq='batch')
  callbacks = [ 
      tensorboard_callback
  ]

  model.fit(
      train_dataset,
      steps_per_epoch=fn_args.train_steps,
      validation_data=eval_dataset,
      validation_steps=fn_args.eval_steps,
      verbose=2,
      callbacks=callbacks)
    
  signatures = {
      'serving_default':
          _get_serve_tf_examples_fn(model,
                                    tf_transform_output).get_concrete_function(
                                        tf.TensorSpec(
                                            shape=[None],
                                            dtype=tf.string,
                                            name='examples')),
  }
  
  model.save(fn_args.serving_model_dir, save_format='tf', signatures=signatures)
  _copy_tensorboard_logs(LOCAL_LOG_DIR, fn_args.serving_model_dir)
    


Overwriting model.py



As of the 0.23 release of TFX, the `Trainer` component only supports passing a single field - `num_steps` - through the `train_args` and `eval_args` arguments. 

In [39]:
trainer = Trainer(
    custom_executor_spec=executor_spec.ExecutorClassSpec(trainer_executor.GenericExecutor),
    module_file=TRAINER_MODULE_FILE,
    transformed_examples=transform.outputs["transformed_examples"],
    schema=schema_gen.outputs["schema"],
    transform_graph=transform.outputs["transform_graph"],
    train_args=trainer_pb2.TrainArgs(num_steps=5000),
    eval_args=trainer_pb2.EvalArgs(num_steps=1000))

### Evaluating trained models with Evaluator
The `Evaluator` component analyzes model performance using the [TensorFlow Model Analysis library](https://www.tensorflow.org/tfx/model_analysis/get_started). It runs inference requests on particular subsets of the test dataset, based on which slices are defined by the developer. Knowing which slices should be analyzed requires domain knowledge of what is important in this particular use case or domain. 

The `Evaluator` can also optionally validate a newly trained model against a previous model. In this lab, you only train one model, so the Evaluator automatically will label the model as "blessed".


<img src=https://github.com/GoogleCloudPlatform/mlops-on-gcp/raw/master/images/Evaluator.png width="400">

You can use the `ResolverNode` to pick the previous model to compare against.  The model resolver is only required if performing model validation in addition to evaluation. In this case we validate against the latest blessed model. If no model has been blessed before (as in this case) the evaluator will make our candidate the first blessed model.

In [40]:
model_resolver = ResolverNode(
      instance_name='latest_blessed_model_resolver',
      resolver_class=latest_blessed_model_resolver.LatestBlessedModelResolver,
      model=Channel(type=Model),
      model_blessing=Channel(type=ModelBlessing))

Configure evaluation metrics and slices.

In [41]:
accuracy_threshold = tfma.MetricThreshold(
                value_threshold=tfma.GenericValueThreshold(
                    lower_bound={'value': 0.5},
                    upper_bound={'value': 0.99}),
                change_threshold=tfma.GenericChangeThreshold(
                    absolute={'value': 0.0001},
                    direction=tfma.MetricDirection.HIGHER_IS_BETTER),
                )

metrics_specs = tfma.MetricsSpec(
                   metrics = [
                       tfma.MetricConfig(class_name='SparseCategoricalAccuracy',
                           threshold=accuracy_threshold),
                       tfma.MetricConfig(class_name='ExampleCount')])

eval_config = tfma.EvalConfig(
    model_specs=[
        tfma.ModelSpec(label_key='Cover_Type')
    ],
    metrics_specs=[metrics_specs],
    slicing_specs=[
        tfma.SlicingSpec(),
        tfma.SlicingSpec(feature_keys=['Wilderness_Area'])
    ]
)
eval_config

model_specs {
  label_key: "Cover_Type"
}
slicing_specs {
}
slicing_specs {
  feature_keys: "Wilderness_Area"
}
metrics_specs {
  metrics {
    class_name: "SparseCategoricalAccuracy"
    threshold {
      value_threshold {
        lower_bound {
          value: 0.5
        }
        upper_bound {
          value: 0.99
        }
      }
      change_threshold {
        absolute {
          value: 0.0001
        }
        direction: HIGHER_IS_BETTER
      }
    }
  }
  metrics {
    class_name: "ExampleCount"
  }
}

In [42]:
model_analyzer = Evaluator(
    examples=example_gen.outputs.examples,
    model=trainer.outputs.model,
    baseline_model=model_resolver.outputs.model,
    eval_config=eval_config
)

### InfraValidator

The `InfraValidator` component acts as an additional early warning layer by validating a candidate model in a sandbox version of its serving infrastructure to prevent an unservable model from being pushed to production. Compared to the `Evaluator` component above which validates a model's performance, the `InfraValidator` component is validating that a model is able to generate predictions from served examples in an environment configured to match production. The config below takes a model and examples, launches the model in a sand-boxed [TensorflowServing](https://www.tensorflow.org/tfx/guide/serving) model server from the latest image in a local docker engine, and optionally checks that the model binary can be loaded and queried before "blessing" it for production.

<img src=https://github.com/GoogleCloudPlatform/mlops-on-gcp/raw/master/images/InfraValidator.png width="400">

In [44]:
infra_validator = InfraValidator(
    model=trainer.outputs['model'],
    examples=example_gen.outputs['examples'],
    serving_spec=infra_validator_pb2.ServingSpec(
        tensorflow_serving=infra_validator_pb2.TensorFlowServing(
            tags=['latest']),
      local_docker=infra_validator_pb2.LocalDockerConfig(),
  ),
    validation_spec=infra_validator_pb2.ValidationSpec(
        max_loading_time_seconds=60,
        num_tries=5,
    ),    
  request_spec=infra_validator_pb2.RequestSpec(
      tensorflow_serving=infra_validator_pb2.TensorFlowServingRequestSpec(),
          num_examples=5,
      )
)

### Deploying models with Pusher

The `Pusher` component checks whether a model has been "blessed", and if so, deploys it by pushing the model to a well known file destination.

<img src=https://github.com/GoogleCloudPlatform/mlops-on-gcp/raw/master/images/Pusher.png width="400">



In [46]:
pusher = Pusher(
    model=trainer.outputs['model'],
    model_blessing=model_analyzer.outputs['blessing'],
    infra_blessing=infra_validator.outputs['blessing'],
    push_destination=pusher_pb2.PushDestination(
        filesystem=pusher_pb2.PushDestination.Filesystem(
            base_directory=SERVING_MODEL_DIR)))

## Running the pipeline

### Create the pipeline

In [47]:
pipeline_name = 'end-to-end'
components = [example_gen, 
              statistics_gen, 
              schema_gen,
              example_validator,
              transform,
              trainer,
              model_resolver,
              model_analyzer,
              infra_validator,
              pusher]

pipeline = Pipeline(
    pipeline_name=pipeline_name,
    pipeline_root=f'{LOCAL_PIPELINE_ROOT}/{pipeline_name}',
    metadata_connection_config=metadata_connection_config,
    components=components
)

### Run the pipeline locally using Beam Runner

In [56]:
BeamDagRunner().run(pipeline)

e_binary>
  child 0, item: large_binary
Wilderness_Area: large_list<item: large_binary>
  child 0, item: large_binary
Aspect: large_list<item: int64>
  child 0, item: int64
Cover_Type: large_list<item: int64>
  child 0, item: int64
Elevation: large_list<item: int64>
  child 0, item: int64
Hillshade_3pm: large_list<item: int64>
  child 0, item: int64
Hillshade_9am: large_list<item: int64>
  child 0, item: int64
Hillshade_Noon: large_list<item: int64>
  child 0, item: int64
Horizontal_Distance_To_Fire_Points: large_list<item: int64>
  child 0, item: int64
Horizontal_Distance_To_Hydrology: large_list<item: int64>
  child 0, item: int64
Horizontal_Distance_To_Roadways: large_list<item: int64>
  child 0, item: int64
Slope: large_list<item: int64>
  child 0, item: int64
Vertical_Distance_To_Hydrology: large_list<item: int64>
  child 0, item: int64, tensor_representations={'Aspect': varlen_sparse_tensor {
  column_name: "Aspect"
}
, 'Hillshade_9am': varlen_sparse_tensor {
  column_name: "Hill

### Examine the artifacts created by the pipeline run

#### Visualize statistics

In [60]:
with metadata.Metadata(connection_config) as store:
    stats_artifacts = store.get_artifacts_by_type(ExampleStatistics.TYPE_NAME)

stats_path = stats_artifacts[-1].uri
train_stats_file = os.path.join(stats_path, 'train', 'stats_tfrecord')
eval_stats_file = os.path.join(stats_path, 'eval', 'stats_tfrecord')
train_stats = tfdv.load_statistics(train_stats_file)
eval_stats = tfdv.load_statistics(eval_stats_file)
tfdv.visualize_statistics(lhs_statistics=eval_stats, rhs_statistics=train_stats,
                          lhs_name='EVAL_DATASET', rhs_name='TRAIN_DATASET')

INFO:absl:MetadataStore with DB connection initialized


#### Visualize model evaluations

In [63]:
with metadata.Metadata(connection_config) as store:
    model_eval_artifacts = store.get_artifacts_by_type(ModelEvaluation.TYPE_NAME)

model_eval_path = model_eval_artifacts[-1].uri
eval_result = tfma.load_eval_result(model_eval_path)
tfma.view.render_slicing_metrics(
    eval_result, slicing_column='Wilderness_Area')

INFO:absl:MetadataStore with DB connection initialized


## Next steps

This concludes your introductory walthrough through TFX pipeline components. In the lab, you used TFX to analyze, understand, and pre-process the dataset and train, analyze, validate, and deploy a multi-class classification model to predict the type of forest cover from cartographic features. You utilized a TFX Interactive Context for prototype development of a TFX pipeline directly in a Jupyter notebook. Next, you worked with the TFDV library to modify your dataset schema to add feature constraints to catch data anamolies that can negatively impact your model's performance. You utilized TFT library for feature proprocessing for consistent feature transformations for your model at training and serving time. Lastly, using the TFMA library, you added model performance constraints to ensure you only push more accurate models than previous runs to production.


The next labs in the series will guide through developing a TFX pipeline, deploying and running the pipeline on **AI Platform Pipelines** and automating the pipeline build and deployment processes with **Cloud Build**.

## License

<font size=-1>Licensed under the Apache License, Version 2.0 (the \"License\");
you may not use this file except in compliance with the License.
You may obtain a copy of the License at [https://www.apache.org/licenses/LICENSE-2.0](https://www.apache.org/licenses/LICENSE-2.0)

Unless required by applicable law or agreed to in writing, software distributed under the License is distributed on an \"AS IS\" BASIS, WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.  See the License for the specific language governing permissions and limitations under the License.</font>